# Exporting Data

Once you are happy with your analysis, you will likely want to export the results. The easiest way to do this is by using the `zarr` format, but `csv`, `anndata`, and `spatialdata` are also supported.

In [4]:
%reload_ext autoreload
%autoreload 2

import spatial_data
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import os
import shutil
xr.set_options(display_style='text')

In [2]:
# loading a test file which we will export later
# notice how easy it is to load the file from a zarr using xarray
ds = xr.load_dataset('../../tests/test_files/test2.zarr')
# ds = xr.load_dataset('/g/huber/projects/CITEseq/CODEX/BNHL_TMA/bhnl_labeled_zarr/166_4_I2_LK_labeled.zarr')
ds

/home/meyerben/meyerben/.conda/envs/spatialproteomics_env/lib/python3.9/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/home/meyerben/meyerben/.conda/envs/spatialproteomics_env/lib/python3.9/site-packages/xarray/backends/plugins.py:159: RuntimeWarning: 'scipy' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)


<xarray.Dataset>
Dimensions:        (cells: 12560, channels: 56, y: 3000, x: 3000, labels: 8,
                    props: 2, features: 4)
Coordinates:
  * cells          (cells) int64 1 2 3 4 5 6 ... 12556 12557 12558 12559 12560
  * channels       (channels) <U11 'DAPI' 'Helios' 'CD10' ... 'CD79a' 'Ki-67'
  * features       (features) <U10 'centroid-0' 'centroid-1' ... '_original_'
  * labels         (labels) int64 1 2 3 4 5 6 7 8
  * props          (props) <U6 '_color' '_name'
  * x              (x) int64 0 1 2 3 4 5 6 ... 2994 2995 2996 2997 2998 2999
  * y              (y) int64 0 1 2 3 4 5 6 ... 2994 2995 2996 2997 2998 2999
Data variables:
    _arcsinh_mean  (cells, channels) float64 3.111 0.0 1.391 ... 1.324 0.4174
    _arcsinh_sum   (cells, channels) float64 8.346 0.0 6.564 ... 6.625 5.224
    _image         (channels, y, x) uint8 4 4 4 4 5 4 4 3 4 ... 2 2 2 2 2 2 2 2
    _labels        (labels, props) object 'C3' ... 'B (PAX5)'
    _obs           (cells, features) float64 613.3 768.4 4.0 ... 8.0 7.0
    _raw_mean      (cells, channels) float64 56.02 0.0 9.426 ... 8.727 2.148
    _raw_sum       (cells, channels) float64 1.053e+04 0.0 ... 1.885e+03 464.0
    _segmentation  (y, x) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0

# Zarr
This is the easiest file format to work with. It allows you to store and load the xarray objects with a single line of code. 

In case there are issues with simply running `ds.to_zarr("your_path.zarr")`, you might need to parse the dtypes correctly. This is a [known issue](https://github.com/pydata/xarray/issues/3476) with xarray and will hopefully be fixed soon.

In [8]:
# parsing as unicode (only necessary if ds.to_zarr() does not work out of the box)
for v in list(ds.coords.keys()):
    if ds.coords[v].dtype == object:
        ds.coords[v] = ds.coords[v].astype("unicode")

for v in list(ds.variables.keys()):
    if ds[v].dtype == object:
        ds[v] = ds[v].astype("unicode")

In [9]:
zarr_path = "tmp.zarr"

# removing the zarr if it exists
if os.path.exists(zarr_path):
    shutil.rmtree(zarr_path)

# exporting as zarr
ds.to_zarr("tmp.zarr")

# Exporting Tables to CSV
Let's say you want to export some tables as csvs. This can be done with pandas.

In [10]:
ds["_arcsinh_mean"]

<xarray.DataArray '_arcsinh_mean' (cells: 12560, channels: 56)>
array([[3.1113318 , 0.        , 1.39104035, ..., 0.42827631, 0.528275  ,
        0.4582596 ],
       [2.80498506, 0.        , 1.16832115, ..., 2.02908251, 0.42634428,
        0.52842933],
       [3.38021961, 0.        , 1.73394497, ..., 0.57174597, 0.72733529,
        0.49741472],
       ...,
       [3.19334124, 0.        , 1.35344852, ..., 0.4051073 , 0.92614581,
        0.43286647],
       [3.43748239, 0.        , 1.5590982 , ..., 0.43428781, 1.39939674,
        0.47313597],
       [3.21632852, 0.        , 1.3602909 , ..., 0.39690411, 1.32359853,
        0.41740323]])
Coordinates:
  * cells     (cells) int64 1 2 3 4 5 6 ... 12555 12556 12557 12558 12559 12560
  * channels  (channels) <U11 'DAPI' 'Helios' 'CD10' ... 'CD79a' 'Ki-67'